In [ ]:
import pandas as pd
import conda_scripts
import os
import conda_scripts.load_datasets as lsd
import numpy as np

# add k values

In [ ]:
tk = pd.read_excel(r"T:\arich\Sonoma_Valley\hydro_stat\other_K_estimates\T and K estimates.xlsx", )
tk = tk.drop([0,7,9,11,12,13,14,16]).loc[:15]



sat = conda_scripts.wiski.wiski.get_stations(site = 'Son',station_name='Son*')
sat.loc[:,'x'] = sat.to_crs(2226).geometry.x
sat.loc[:,'y'] = sat.to_crs(2226).geometry.y

sat = sat.loc[:,['station_name','x','y']]

assert tk.station_name.duplicated().any()

tk = pd.merge(tk, sat, left_on = 'station_name', right_on = 'station_name',how = 'left')

In [ ]:
template_ws = "pestaws_jam"

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(template_ws)

In [ ]:
[1,2,3,4][slice(2,2+1,1)]

In [ ]:
c = tk.x.isnull()

tk.loc[c,'x'] = tk.loc[c,'STP_X']
tk.loc[c,'y'] = tk.loc[c,'STP_Y']



tk.loc[:,'i'] = tk.apply(lambda x: ml.modelgrid.intersect(x['x'],x['y'])[0],axis = 1)
tk.loc[:,'j'] = tk.apply(lambda x: ml.modelgrid.intersect(x['x'],x['y'])[1],axis = 1)

hk = [np.genfromtxt(os.path.join(template_ws,'pp2024_out',f'hk{i}.txt')) for i in range(1,7)]
hk = np.stack(hk)

for ind,row in tk.iterrows():

    bot = ml.dis.botm.array[:,row['i'],row['j']]
    top = ml.dis.top.array[row['i'],row['j']]
    print(row['station_name'])
    print('top',top,'bot', bot)
    print(row['screen'])
    t = float(row['screen'].split('-')[0]) if '-' in row['screen'] else 290
    b = float(row['screen'].split('-')[1]) if '-' in row['screen'] else 300
    t = top-t
    b = top-b
    laytop = np.argmax(t>bot)
    laybot = np.argmax(b>bot)
    if min(bot)>t:
        laytop =5
        laybot = 5
    print(t,b, laytop,laybot)

    tk.loc[ind,'laytop'] = laytop
    tk.loc[ind,'laybot'] = laybot
    tk.loc[ind,'screen_top'] = t
    tk.loc[ind,'screen_bot'] = b
    tk.loc[ind,[f'laybot_{i+1}' for i in range(6)]] = bot
    print(f"hk observed = {row['k_PEST']}")
    avg = hk[slice(laytop,laybot+1,1),row['i'],row['j']]
    print(f"average--{avg}")

    print('\n'*2)
    tk.loc[ind,'hk_cur_model'] = np.mean(avg)
    tk.loc[ind,'weight'] = 1/(1+laybot-laytop)


In [ ]:
tk.columns

In [ ]:
ax = tk.plot.scatter(x = 'k_PEST', y = 'hk_cur_model') 
ax.set_xlabel('Observed K')
ax.set_ylabel('Simulated K')
# Set the aspect ratio to be equal
ax.set_aspect('equal', adjustable='datalim')
ax.grid(True)

In [ ]:
tk.to_excel('hk_estimates/hk_estimates.xlsx')

In [ ]:
tab = pd.read_clipboard()

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
def HK_extract(workspace):
    '''extract K values from observed hk values from model'''

    infile = os.path.join(workspace, 'hk_estimates', "hk_estimates_for_pest.xlsx")
    outfile = os.path.join(workspace, 'hk_estimates', "hk_estimates_for_pest_simulated_K.csv")
    obs_vals = pd.read_excel(os.path.join(workspace, 'hk_estimates', "hk_estimates_for_pest.xlsx"),index_col = 0)
    
    hk = [np.genfromtxt(os.path.join(workspace,'pp2024_out',f'hk{i}.txt')) for i in range(1,7)]
    hk = np.stack(hk)
    print('extracting HK values from model\n')
    for ind,row in obs_vals.iterrows():
    
        laytop = int(row['laytop'])
        laybot = int(row['laybot'])
        
        
        avg = hk[slice(laytop,laybot+1,1),row['i'],row['j']]
        obs_vals.loc[ind,'hk_pest'] = np.mean(avg)
        
        print(f"wellname:{row['well']}\n\tlaytop - {laybot}, laybot-{laybot}\n\tlayer values:--\n\t\t{avg}\n\tactual value\n\t\t{np.mean(avg):.3g}\n")

    obs_vals.to_csv(outfile)


    return obs_vals
    
obs = HK_extract(workspace)

In [ ]:
obs